# Introdução

In [ ]:
import os
import glob
import pandas as pd

## Pontos de Monitoramento

A partir da lista dos pontos de monitoramento, é possível extraír a primeira letra de cada um dos pontos demonitoramento.

A partir dessa lista, é possível iterar e obter tudo.

In [ ]:
# Tabela Pontos de Monitoramento
df = pd.read_csv(
    os.path.join('data', 'tab_pontos_monitoramento.csv'),
)

# Results
df.head()

In [ ]:
list_letter = list(set(df['Cód.Ponto'].astype(str).str[0]))
list_letter.sort()
list_letter

In [ ]:
# Filter
#list_letter = list_letter[5:6]
list_letter

## Merge Data

In [ ]:
all_data = pd.DataFrame()
for letter in list_letter:
    for f in glob.glob(os.path.join(os.getcwd(), 'data', letter, '*.xlsx')):
        try:
            print(f)
            df = pd.read_excel(f)
            all_data = all_data.append(df, ignore_index=True)
        except OSError as e:
            print(e)

# Results
df = all_data
df.head()

<br>

## Rename Columns

In [ ]:
# Dictonary to Rename
dict_columns = {
    'Código Ponto'           : 'cod_ponto',
    'Cod_Interaguas'         : 'cod_interaguas',
    'Tipo Rede'              : 'tipo_rede',
    'Status Ponto'           : 'status_ponto',
    'Data Coleta'            : 'data_coleta',
    'Hora Coleta'            : 'hora_coleta',
    'Parametro'              : 'parametro',
    'Sinal'                  : 'sinal',
    'Valor'                  : 'valor',
    'Unidade'                : 'unidade',
    'Tipo Parâmetro'         : 'tipo_parametro',
    'Sistema Hídrico'        : 'sistema_hidrico',
    'Tipo de Sistema Hídrico': 'sistema_hidrico_tipo',
    'CLASSE'                 : 'classe',
    'Inicio Operação'        : 'data_inicio_operacao',
    'Fim Operação'           : 'data_fim_operacao',
    'Período DE'             : 'data_inicio_query',
    'Período ATE'            : 'data_fim_query',
    'Latitude'               : 'latitude_dms',
    'Longitude'              : 'longitude_dms',
    'Altitude'               : 'altitude',
    'Localização'            : 'localizacao',
    'Captação'               : 'captacao',
    'Município'              : 'municipio',
    'UF'                     : 'estado',
    'UGRHI'                  : 'ugrhi',
}

# Rename e Reorder
df = df.rename(columns=dict_columns)
df = df[[v for k, v in dict_columns.items()]]

# Columns
print(list(df.columns))

# Results
df.head()

## Ajusta Datas

In [ ]:
# Ajuste Dates
df['data_coleta']          = pd.to_datetime(df['data_coleta'], format='%d/%m/%Y')
df['hora_coleta']          = pd.to_datetime(df['hora_coleta'], format='%H:%M').dt.time
df['data_inicio_operacao'] = pd.to_datetime(df['data_inicio_operacao'], format='%d/%m/%Y')
df['data_fim_operacao']    = pd.to_datetime(df['data_fim_operacao'], format='%d/%m/%Y')
df['data_inicio_query']    = pd.to_datetime(df['data_inicio_query'], format='%d/%m/%Y')
df['data_fim_query']       = pd.to_datetime(df['data_fim_query'], format='%d/%m/%Y')

## Ajusta Coordenadas

In [ ]:
def dms2dd(coord_dms):
    coord_dms = coord_dms.strip()
    try:
        coord_deg = float(coord_dms.split(' ', maxsplit=2)[0])
        coord_min = float(coord_dms.split(' ', maxsplit=2)[1])
        coord_sec = float(coord_dms.split(' ', maxsplit=2)[2])
        coord_dd = coord_deg + (coord_min/60) + (coord_sec/3600)
        coord_dd = coord_dd * -1
    except:
        coord_dd = 0
    return coord_dd

dms2dd('22 13 52')

In [ ]:
# Coordenadas
df.loc[:, 'latitude_dd'] = df['latitude_dms'].astype(str).apply(lambda x: dms2dd(x))
df.loc[:, 'longitude_dd'] = df['longitude_dms'].astype(str).apply(lambda x: dms2dd(x))

## Resultados

In [ ]:
# Export
df.to_csv(
    os.path.join('data', 'tab_analises.csv'),
    index=False,
)